In [2]:
import tensorflow as tf

In [3]:
from mtcnn import MTCNN
import cv2
detector = MTCNN()
#Load a videopip TensorFlow

In [5]:
video_capture = cv2.VideoCapture(0)
 
while (True):
    ret, frame = video_capture.read()
    frame = cv2.resize(frame, (600, 400))
    boxes = detector.detect_faces(frame)
    if boxes:
 
        box = boxes[0]['box']
        conf = boxes[0]['confidence']
        x, y, w, h = box[0], box[1], box[2], box[3]
 
        if conf > 0.5:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 255), 3)
 
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
 
video_capture.release()
cv2.destroyAllWindows()

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [5]:
pwd

'C:\\Users\\Melody\\Documents\\workspace\\computer vision'

In [161]:
imagePaths = list(paths.list_images('dataset-20201016T175141Z-001/dataset'))
data = []
labels = []

In [162]:
imagePaths[0].split(os.path.sep)

['dataset-20201016T175141Z-001/dataset', 'without_mask', '0.jpg']

In [163]:
for imagePath in imagePaths:
	label = imagePath.split(os.path.sep)[-2]
	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	image = preprocess_input(image)
	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [164]:
len(labels)

3957

In [165]:
labels.shape

(3957,)

In [167]:
for i in range(10):
    cv2.imshow("Tobago", data[-100+i]) 

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [168]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_shape=(224, 224, 3))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.3)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [169]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [170]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [171]:
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)
#construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=40,
	zoom_range=0.3,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [172]:
##calssbacks:

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='saved_model/',
                                                        save_best_only=True,
                                                        monitor='accuracy',
                                                        save_weights_only=True,
                                                        mode='max',
                                                        save_freq='epoch')

earlyStopping_callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy',
                                                         patience=15,
                                                         mode='max')

In [212]:
trainX.shape

(3165, 224, 224, 3)

In [173]:
INIT_LR = 1e-4
EPOCHS = 50
BS = 32
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS,
    callbacks=[checkpoint_callback,earlyStopping_callback]            )

[INFO] compiling model...
[INFO] training head...
Epoch 1/50
98/98 [==============================] - 129s 1s/step - loss: 0.4081 - accuracy: 0.8174 - val_loss: 0.1812 - val_accuracy: 0.9520
Epoch 2/50
98/98 [==============================] - 118s 1s/step - loss: 0.1880 - accuracy: 0.9365 - val_loss: 0.1365 - val_accuracy: 0.9558
Epoch 3/50
98/98 [==============================] - 120s 1s/step - loss: 0.1637 - accuracy: 0.9403 - val_loss: 0.1201 - val_accuracy: 0.9634
Epoch 4/50
98/98 [==============================] - 118s 1s/step - loss: 0.1278 - accuracy: 0.9582 - val_loss: 0.1156 - val_accuracy: 0.9659
Epoch 5/50
98/98 [==============================] - 124s 1s/step - loss: 0.1223 - accuracy: 0.9566 - val_loss: 0.1031 - val_accuracy: 0.9659
Epoch 6/50
98/98 [==============================] - 122s 1s/step - loss: 0.1168 - accuracy: 0.9620 - val_loss: 0.0978 - val_accuracy: 0.9659
Epoch 7/50
98/98 [==============================] - 122s 1s/step - loss: 0.1043 - accuracy: 0.9633 - val

In [14]:
#model = load_model("saved_model/mask_recog_ver2.h5")

In [214]:
gray.shape

(480, 640)

In [16]:
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
 
cascPath = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPath)
#model = load_model("saved_model/mask_recog_ver2.h5")
 
video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags = cv2.CASCADE_SCALE_IMAGE|cv2.CASCADE_FIND_BIGGEST_OBJECT|cv2.CASCADE_DO_ROUGH_SEARCH)
    faces_list=[]
    preds=[]
    for (x, y, w, h) in faces:
        face_frame = frame[y:y+h,x:x+w]
        face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
        face_frame = cv2.resize(face_frame, (224, 224))
        face_frame = img_to_array(face_frame)
        face_frame = np.expand_dims(face_frame, axis=0)
        face_frame =  preprocess_input(face_frame)
        faces_list.append(face_frame)
        if len(faces_list)>0:
            preds = model.predict(faces_list)
        for pred in preds:
            (mask, withoutMask) = pred
        #label = "Mask" if mask > withoutMask else "No Mask"
        label = "Is Mask ->" + str(mask)
        #color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        color = (0, 255, 0) if mask>0.985  else (0, 0, 255)
        #color = (255,255,255)
        #label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (x, y- 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
 
        cv2.rectangle(frame, (x, y), (x + w, y + h),color, 2)
        # Display the resulting frame
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) ==13:
        break
video_capture.release()
cv2.destroyAllWindows()

In [8]:
#model.save(filepath='saved_model/improvement_model.h5')

In [6]:
model = tf.keras.models.load_model('saved_model/improvement_model.h5')

In [9]:
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
 
cascPath = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier('haarcascads/frontalEyes35x16.xml')
profileFace = cv2.CascadeClassifier('haarcascads/haarcascade_eye.xml')
#model = load_model("saved_model/mask_recog_ver2.h5")

video_capture = cv2.VideoCapture(0)
while True:
    
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    
    alpha = 1.2 # Contrast control (1.0-3.0)
    beta = 20 # Brightness control (0-100)

    frame = cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(45, 45),
                                         flags = cv2.CASCADE_SCALE_IMAGE|cv2.CASCADE_FIND_BIGGEST_OBJECT|cv2.CASCADE_DO_ROUGH_SEARCH)
    
    h_dup_by = 3
    w_dup_by = 1
    
    if(len(faces)==0):
        faces = profileFace.detectMultiScale(gray,
                                             scaleFactor=1.1,
                                             minNeighbors=5,
                                             minSize=(45, 45),
                                             flags = cv2.CASCADE_SCALE_IMAGE|cv2.CASCADE_FIND_BIGGEST_OBJECT|cv2.CASCADE_DO_ROUGH_SEARCH)

    
        h_dup_by = 3
        w_dup_by = 3
    
    faces_list=[]
    preds=[]
    for (x, y, w, h) in faces:
        y=y-50
        if (y<=0):
            y=0
        h=h*h_dup_by
        w=w*w_dup_by
        face_frame = frame[y:y+h,x:x+w]
        face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
        face_frame = cv2.resize(face_frame, (224, 224))
        face_frame = img_to_array(face_frame)
        face_frame = np.expand_dims(face_frame, axis=0)
        face_frame =  preprocess_input(face_frame)
        faces_list.append(face_frame)
        if len(faces_list)>0:
            preds = model.predict(faces_list)
        for pred in preds:
            (mask, withoutMask) = pred
        label = "Mask  " if mask > 0.99 else "No Mask  "
        label = label + str(round(mask,4))
        #label = "Mask" + str(mask)
        #color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        color = (0, 255, 0) if mask>0.99  else (0, 0, 255)
        #color = (255,255,255)
        #label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (x, y- 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
 
        cv2.rectangle(frame, (x, y), (x + w, y + h),color, 2)
        # Display the resulting frame
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) ==13:
        break
video_capture.release()
cv2.destroyAllWindows()

#### from video

In [8]:
import cv2
import os

# for _, __, files in os.walk("."):
#     for file in files:
#         print file
# print ""

# cap = cv2.VideoCapture("office video/poc.mp4")

# cap.set(cv2.CAP_PROP_FPS, 1000)
# i = 0
# while  i < 500:
#     i += 1
#     #if (i%3==0):
#     ret, frame = cap.read()
#     frame=cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
#     #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     cv2.imshow('frame', frame)
#     if cv2.waitKey(15) & 0xFF == ord('q'):
#         break
# cap.release()
# cv2.destroyAllWindows()

In [32]:
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
 
cascPath = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
#faceCascade = cv2.CascadeClassifier('haarcascads/frontalEyes35x16.xml')
faceCascade = cv2.CascadeClassifier('haarcascads/haarcascade_upperbody.xml')
#profileFace = cv2.CascadeClassifier('haarcascads/haarcascade_eye.xml')
#model = load_model("saved_model/mask_recog_ver2.h5")




cap = cv2.VideoCapture("office video/poc.mp4")

cap.set(cv2.CAP_PROP_FPS, 1000)
i = 0
# while  i < 500:
#     i += 1
#     #if (i%3==0):
#     ret, frame = cap.read()
#     frame=cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     cv2.imshow('frame', frame)
#     if cv2.waitKey(15) & 0xFF == ord('q'):
#         break
# cap.release()
# cv2.destroyAllWindows()
 
# video_capture = cv2.VideoCapture(0)
while  i < 500:
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame=cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    
    alpha = 1.1 # Contrast control (1.0-3.0)
    beta = 10 # Brightness control (0-100)

    frame = cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags = cv2.CASCADE_SCALE_IMAGE|cv2.CASCADE_FIND_BIGGEST_OBJECT|cv2.CASCADE_DO_ROUGH_SEARCH)
    
    h_dup_by = 3
    w_dup_by = 1
    
    if(len(faces)==0):
        faces = profileFace.detectMultiScale(gray,
                                             scaleFactor=1.1,
                                             minNeighbors=5,
                                             minSize=(60, 60),
                                             flags = cv2.CASCADE_SCALE_IMAGE|cv2.CASCADE_FIND_BIGGEST_OBJECT|cv2.CASCADE_DO_ROUGH_SEARCH)

    
        h_dup_by = 3
        w_dup_by = 3
    
    faces_list=[]
    preds=[]
    for (x, y, w, h) in faces:
        
        h=h*h_dup_by
        w=w*w_dup_by
        face_frame = frame[y:y+h,x:x+w]
        face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
        face_frame = cv2.resize(face_frame, (224, 224))
        face_frame = img_to_array(face_frame)
        face_frame = np.expand_dims(face_frame, axis=0)
        face_frame =  preprocess_input(face_frame)
        faces_list.append(face_frame)
        if len(faces_list)>0:
            preds = model.predict(faces_list)
        for pred in preds:
            (mask, withoutMask) = pred
        label = "Mask  " if mask > 0.995 else "No Mask  "
        label = label + str(round(mask,4))
        #label = "Mask" + str(mask)
        #color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        color = (0, 255, 0) if mask>0.985  else (0, 0, 255)
        #color = (255,255,255)
        #label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (x, y- 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
 
        cv2.rectangle(frame, (x, y), (x + w, y + h),color, 2)
        # Display the resulting frame
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) ==13:
        break
cap.release()
cv2.destroyAllWindows()

#### evaluate model

In [236]:
import sklearn
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score()

In [237]:
model.predict(trainX)

array([[1.6358974e-05, 9.9998367e-01],
       [9.6657068e-01, 3.3429299e-02],
       [9.9977833e-01, 2.2165888e-04],
       ...,
       [9.9960214e-01, 3.9787785e-04],
       [1.8457719e-03, 9.9815422e-01],
       [9.9991965e-01, 8.0319129e-05]], dtype=float32)

In [29]:
import numpy as np
import cv2
 
# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

cv2.startWindowThread()

# open webcam video stream
cap = cv2.VideoCapture(0)

# the output will be written to output.avi
out = cv2.VideoWriter(
    'output.avi',
    cv2.VideoWriter_fourcc(*'MJPG'),
    15.,
    (640,480))

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # resizing for faster detection
    frame = cv2.resize(frame, (640, 480))
    # using a greyscale picture, also for faster detection
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

    # detect people in the image
    # returns the bounding boxes for the detected objects
    boxes, weights = hog.detectMultiScale(frame, winStride=(8,8) )

    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])

    for (xA, yA, xB, yB) in boxes:
        # display the detected boxes in the colour picture
        cv2.rectangle(frame, (xA, yA), (xB, yB),
                          (0, 255, 0), 2)
    
    # Write the output video 
    out.write(frame.astype('uint8'))
    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
# and release the output
out.release()
# finally, close the window
cv2.destroyAllWindows()
cv2.waitKey(1)

-1